In [ ]:
# For Prediction of Drug-Induced Liver Injuries using ML/AI Methods.
# Neural Network, Bayesian Neural Network (Basic), NuSVC, and Probabilistic Odds Logistic Regression (POLR) Model
# Advanced BNN is seperated written in the following file

#*******************************************
# Developed by: Ashok K Sharma
# Completed on: May 25th - June 6th, 2023
#******************************************

In [ ]:
# Function to count and sort the unique elements in the input data
def sort_map(input_data):
    # Get the unique elements and their counts
    unique, counts = np.unique(input_data, return_counts=True)
    # Create a dictionary mapping each unique element to its count
    count_dict = dict(zip(unique, counts))
    # Sort the dictionary and return it
    return sorted(count_dict.items())

# Read the train and test datasets
train_data = pd.read_csv("Code_check/1-s2.0-S2468111320300438-mmc8.csv")
test_data = pd.read_csv("Code_check/1-s2.0-S2468111320300438-mmc10.csv")

# Select the desired features from the datasets
features = ['ClogP', 'BSEP', 'Glu', 'Glu_Gal', 'THLE', 'HepG2', 'Fsp3', 'log10cmax']
X_train_df = train_data[features]
X_test_df = test_data[features]

# Initialize a StandardScaler to standardize the features to have mean=0 and variance=1
scaler = StandardScaler()

# Fit the StandardScaler on the training data and transform both training and test data
scaler.fit(X_train_df)
X_train = pd.DataFrame(scaler.transform(X_train_df), columns=X_train_df.columns)
X_test = pd.DataFrame(scaler.transform(X_test_df), columns=X_test_df.columns)

# Select the target variable
target = ['dili_sev']
Y_train = (train_data[target].values.astype(int) - 1)
Y_test = (test_data[target].values.astype(int) - 1)

# Apply the sort_map function on the target variable
sorted_train_output = sort_map(Y_train)
sorted_test_output = sort_map(Y_test)

# Print the sorted output
print(sorted_train_output)
print(sorted_test_output)

In [ ]:
#-- This segment Specifically Needs to Run NN, and BNN
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert numpy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train.values).float().to(device)
Y_train = torch.from_numpy(Y_train.flatten()).long().to(device)
X_test = torch.from_numpy(X_test.values).float().to(device)
Y_test = torch.from_numpy(Y_test.flatten()).long().to(device)

### 1. Create a Neural Network Model

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Define the Feedforward Neural Network
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# set_seed(123), set_seed(34), set_seed(91)
set_seed(91)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define dimensions
input_dim = X_train.shape[1] # It should match with the number of features in the dataset
hidden_dim = 128 # You can change this
output_dim = len(np.unique(Y_train)) # It should match with the number of unique target values

# Create the model
model = FFNN(input_dim, hidden_dim, output_dim).to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


# Train the model
epochs = 450 # You can change this
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    loss.backward()
    optimizer.step()

    # Calculate train accuracy and confusion matrix in each epoch
    _, predicted_nn = torch.max(outputs.data, 1)
    train_accuracy = 100 * (predicted_nn == Y_train).sum().item() / len(Y_train)
    conf_matrix = confusion_matrix(Y_train.cpu(), predicted_nn.cpu())


#     if (epoch+1) % 10 == 0:
#         print(f'Epoch {epoch+1}, Loss: {loss.item()}, Train Accuracy: {train_accuracy}%')

print('Confusion Matrix:')
print(conf_matrix)

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted_nn = torch.max(outputs.data, 1)
    correct = (predicted_nn == Y_test).sum().item()
    conf_matrix = confusion_matrix(Y_test.cpu(), predicted_nn.cpu())
    print(f'Accuracy of the model on the test set: {100 * correct / len(Y_test)}%')
    print('Confusion Matrix:')
    print(conf_matrix)

In [ ]:
# Print Actual labels and Predictions
Y_test, predicted_nn

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    # Convert the output probabilities to predicted class
    _, predicted_nn = torch.max(outputs.data, 1)
    correct = (predicted_nn == Y_test).sum().item()

    # Compute the probabilities via softmax
    probabilities_nn = torch.nn.functional.softmax(outputs, dim=1)
    print('Predicted Probabilities:', probabilities_nn)
    print('Predicted :', predicted_nn)
    print(f'Accuracy of the model on the test set: {100 * correct / len(Y_test)}%')

In [ ]:
#----- Save Different Performance Matrices
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

# Compute precision, accuracy, F1-score, and MCC for the predicted and true labels
precision = precision_score(Y_test, predicted_nn, average='macro')
accuracy = accuracy_score(Y_test, predicted_nn)
f1_score = f1_score(Y_test, predicted_nn, average='macro')
mcc = matthews_corrcoef(Y_test, predicted_nn)

# Calculate sensitivity and specificity using confusion matrix
cm = confusion_matrix(Y_test, predicted_nn)

# Calculate sensitivity and specificity for each class
sensitivity = {}
specificity = {}

for i in range(cm.shape[0]):
    tp = cm[i, i]
    fn = sum(cm[i, :]) - tp
    fp = sum(cm[:, i]) - tp
    tn = cm.sum() - (tp + fn + fp)

    sensitivity[i] = tp / (tp + fn)
    specificity[i] = tn / (tn + fp)

# Calculate overall sensitivity and specificity
overall_sensitivity = sum(sensitivity.values()) / len(sensitivity)
overall_specificity = sum(specificity.values()) / len(specificity)

# Print the performance metrics to the console
print("***************** MODEL PERFORMANCE: ****************")
print("Confusion Matrix:")
print(cm)
print("Sensitivity:")
for key, value in sensitivity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Specificity:")
for key, value in specificity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Overall Sensitivity: {:.2f}".format(overall_sensitivity))
print("Overall Specificity: {:.2f}".format(overall_specificity))

print("Precision: {:.2f}".format(precision))
print("Accuracy: {:.2f}".format(accuracy))
print("F1-score: {:.2f}".format(f1_score))
print("MCC: {:.2f}".format(mcc))

In [ ]:
###### To Save Neural Network Outputs- Original Labels, Predicted lables, Probabilities, Drug Names Along with Variable Scaler values
#!mkdir = ML_results # One time Taks and Done
merged_array_nn = np.column_stack((Y_test, predicted_nn))
merged_array_nn_df = pd.DataFrame(data=merged_array_nn, columns=['Y_test', 'Y_predicted'])
#print(merged_array_nn_df)

# Convert predicted probabilities to a NumPy array
probs_array_nn = probabilities_nn.detach().numpy()
# Create a DataFrame from the NumPy array
probs_df_nn = pd.DataFrame(probs_array_nn, columns=['Class 0', 'Class 1', 'Class 2'])  # Add column names as per your classes
# Print the DataFrame
#print(probs_df_nn)

#-- Get the Drug names from Test Dataset and Scaler values of All variables
drug_names = test_data[['Drug']]
numpy_array = X_test.numpy()
X_test_scaler_df = pd.DataFrame(numpy_array, columns =X_test_df.columns)
#X_test_scaler_df

#--- Merge Y_Test_Labels, Y_preds, Y_preds_Proabilites and test Dataset information also
concatenated_df_nn = pd.concat([merged_array_nn_df, probs_df_nn, drug_names, X_test_scaler_df], axis=1)
print (concatenated_df_nn)

#concatenated_df_nn.to_csv('ML_results/Neural_nets_Preds.csv', index=False)  # Specify the desired file name and path

### 2. Create Bayesian Neural Network

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    pyro.set_rng_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define the Bayesian Feedforward Neural Network
class BayesianFFNN(pyro.nn.PyroModule):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BayesianFFNN, self).__init__()
        self.fc1 = pyro.nn.PyroModule[nn.Linear](input_dim, hidden_dim)
        self.fc1.weight = pyro.nn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim, input_dim]).to_event(2))
        self.fc1.bias = pyro.nn.PyroSample(dist.Normal(0., 1.).expand([hidden_dim]).to_event(1))
        self.fc3 = pyro.nn.PyroModule[nn.Linear](hidden_dim, output_dim)
        self.fc3.weight = pyro.nn.PyroSample(dist.Normal(0., 1.).expand([output_dim, hidden_dim]).to_event(2))
        self.fc3.bias = pyro.nn.PyroSample(dist.Normal(0., 1.).expand([output_dim]).to_event(1))

        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        out = self.relu(self.fc1(x))
        out = self.fc3(out)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Categorical(logits=out), obs=y)
        return out

# set_seed(25, 234) ~ 40%
# set_seed(234) ~ 45%
# set_seed(91) ~54%

set_seed(91)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define dimensions
input_dim = X_train.shape[1]
hidden_dim = 32
output_dim = len(np.unique(Y_train))

# Create the model
model = BayesianFFNN(input_dim, hidden_dim, output_dim).to(device)
guide = pyro.infer.autoguide.AutoDiagonalNormal(model)

# Define the loss function and the optimizer
optimizer = Adam({"lr": 0.15})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())



# Train the model
epochs = 73
for epoch in range(epochs):
    model.train()

    loss = svi.step(X_train, Y_train)

    # Calculate train accuracy and confusion matrix in each epoch
    predicted = torch.argmax(model(X_train), dim=-1, keepdim=True)
    train_accuracy = 100 * (predicted.squeeze() == Y_train).sum().item() / len(Y_train)
    conf_matrix = confusion_matrix(Y_train.cpu(), predicted.cpu().squeeze())

    if (epoch+1) % 50 == 0:
       print(f'Epoch {epoch+1}, Loss: {loss}, Train Accuracy: {train_accuracy}%')

#     print('Confusion Matrix:')
#     print(conf_matrix)

    # Evaluate the model

    model.eval()
    with torch.no_grad():
        num_samples = 100
        sampled_models = [guide(X_test) for _ in range(num_samples)]
        outputs = torch.stack([model.forward(X_test) for _ in sampled_models]).mean(0)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == Y_test).sum().item()
        conf_matrix = confusion_matrix(Y_test.cpu(), predicted.cpu())
        print(f'Accuracy of the model on the test set: {epoch+1} {100 * correct / len(Y_test)}%')
        print('Confusion Matrix:')
        print(conf_matrix)

In [ ]:
# Print Actual Labels and Predictions.
Y_test, predicted

In [ ]:
probabilities = torch.nn.functional.softmax(outputs, dim=1)
print('Prediction Probabilities:')
print(probabilities)

In [ ]:
#----- Save Different Performance Matrices
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

# Compute precision, accuracy, F1-score, and MCC for the predicted and true labels
precision = precision_score(Y_test, predicted, average='macro')
accuracy = accuracy_score(Y_test, predicted)
f1_score = f1_score(Y_test, predicted, average='macro')
mcc = matthews_corrcoef(Y_test, predicted)

# Calculate sensitivity and specificity using confusion matrix
cm = confusion_matrix(Y_test, predicted)

# Calculate sensitivity and specificity for each class
sensitivity = {}
specificity = {}

for i in range(cm.shape[0]):
    tp = cm[i, i]
    fn = sum(cm[i, :]) - tp
    fp = sum(cm[:, i]) - tp
    tn = cm.sum() - (tp + fn + fp)

    sensitivity[i] = tp / (tp + fn)
    specificity[i] = tn / (tn + fp)

# Calculate overall sensitivity and specificity
overall_sensitivity = sum(sensitivity.values()) / len(sensitivity)
overall_specificity = sum(specificity.values()) / len(specificity)

# Print the performance metrics to the console
print("**************** MODEL PERFORMANCE: Bayesian Neural Network****************")
print("Confusion Matrix:")
print(cm)
print("Sensitivity:")
for key, value in sensitivity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Specificity:")
for key, value in specificity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Overall Sensitivity: {:.2f}".format(overall_sensitivity))
print("Overall Specificity: {:.2f}".format(overall_specificity))

print("Precision: {:.2f}".format(precision))
print("Accuracy: {:.2f}".format(accuracy))
print("F1-score: {:.2f}".format(f1_score))
print("MCC: {:.2f}".format(mcc))

In [ ]:
###### To Save Bayesian Neural Network Outputs- Original Labels, Predicted lables, Probabilities, Drug Names Along with Variable Scaler values
# Save the Bayesian Neural Network Results
merged_array = np.column_stack((Y_test, predicted))
df = pd.DataFrame(data=merged_array, columns=['Y_test', 'Y_predicted'])
#print (df)

# Convert predicted probabilities to a NumPy array
probs_array = probabilities.detach().numpy()
# Create a DataFrame from the NumPy array
probs_df = pd.DataFrame(probs_array, columns=['Class 0', 'Class 1', 'Class 2'])  # Add column names as per your classes
# Print the DataFrame
#print(probs_df)

#-- Get the Drug names from Test Dataset and Scaler values of All variables
drug_names = test_data[['Drug']]
numpy_array = X_test.numpy()
X_test_scaler_df = pd.DataFrame(numpy_array, columns = X_test_df.columns)
#X_test_scaler_df

#--- Merge Y_Test_Labels, Y_preds, Y_preds_Proabilites and test Dataset information also
concatenated_df = pd.concat([df, probs_df, drug_names, X_test_scaler_df], axis=1)
print (concatenated_df)

#concatenated_df.to_csv('ML_results/Bayesian_Neural_nets_Preds.csv', index=False)  # Specify the desired file name and path

### 3.Try Other Machine Learning NuSVC (Selected via comparing Multiple Machine Learning Based Models:
### Projects/Project_DILI/semenova_data/multiML_AZ_data.ipynb)

In [ ]:
#NuSVC was selected because it was performing better than other ML methods
def sort_map(input_data):
    # Get the unique elements and their counts
    unique, counts = np.unique(input_data, return_counts=True)
    # Create a dictionary mapping each unique element to its count
    count_dict = dict(zip(unique, counts))
    # Sort the dictionary and return it
    return sorted(count_dict.items())

# Read DataSET Again becuase for NN and BNN data was converted in to Tensor Objects
# Read the train and test datasets
train_data = pd.read_csv("Code_check/1-s2.0-S2468111320300438-mmc8.csv")
test_data = pd.read_csv("Code_check/1-s2.0-S2468111320300438-mmc10.csv")

# Select the desired features from the datasets
features = ['ClogP', 'BSEP', 'Glu', 'Glu_Gal', 'THLE', 'HepG2', 'Fsp3', 'log10cmax']
X_train_df = train_data[features]
X_test_df = test_data[features]

# Initialize a StandardScaler to standardize the features to have mean=0 and variance=1
scaler = StandardScaler()

# Fit the StandardScaler on the training data and transform both training and test data
scaler.fit(X_train_df)
X_train = pd.DataFrame(scaler.transform(X_train_df), columns=X_train_df.columns)
X_test = pd.DataFrame(scaler.transform(X_test_df), columns=X_test_df.columns)

# Select the target variable
target = ['dili_sev']
Y_train = (train_data[target].values.astype(int) - 1)
Y_test = (test_data[target].values.astype(int) - 1)

# Apply the sort_map function on the target variable
sorted_train_output = sort_map(Y_train)
sorted_test_output = sort_map(Y_test)

# Print the sorted output
print(sorted_train_output)
print(sorted_test_output)


In [ ]:
#import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.svm import NuSVC
check = make_pipeline(StandardScaler(), NuSVC())
check.fit(X_train, Y_train)
Pipeline(steps=[('standardscaler', StandardScaler()), ('nusvc', NuSVC())])

Y_train = Y_train.ravel()
Y_test = Y_test.ravel()

#set_seed(110)

#-- Check Performance of NuSVC using Multiple Kernels
#******************************* LINEAR Kernal
# Create a NuSVC object with a linear kernel
model_linear = NuSVC(kernel='linear', probability=True)
# Fit the classifier to the training data
model_linear.fit(X_train, Y_train)

#----- Get Cross Validation Performances
cv_scores_linear = cross_val_score(model_linear, X_train, Y_train, cv=5, scoring='accuracy')
# Print the cross-validation scores
print("Cross-Validation Scores: Linear Kernal", cv_scores_linear)
print("Mean Cross-Validation Score: Linear Kernal", cv_scores_linear.mean())

# Get the predictions on the test data
predictions_linear = model_linear.predict(X_test)
#--- Get prediction probabilites
probabilites_linear = model_linear.predict_proba(X_test)
# Get the performance metrics
metrics_linear = model_linear.score(X_test, Y_test)
# Print the performance metrics
print('Linear Kernel - Accuracy on Test Data')
print(metrics_linear)
from sklearn.metrics import confusion_matrix
conf_mat_linear = confusion_matrix(Y_test, predictions_linear)
print('Linera Kernel Confusion Matrix')
print(conf_mat_linear)

#******************************* RBF Kernal
# Create a NuSVC object with an RBF kernel
model_rbf = NuSVC(kernel='rbf', probability=True)
# Fit the classifier to the training data
model_rbf.fit(X_train, Y_train)

#----- Get Cross Validation Performances
cv_scores_rbf = cross_val_score(model_rbf, X_train, Y_train, cv=5, scoring='accuracy')
# Print the cross-validation scores
print("Cross-Validation Scores: RBF Kernal", cv_scores_rbf)
print("Mean Cross-Validation Score: RBF Kernal", cv_scores_rbf.mean())

# Get the predictions on the test data
predictions_rbf = model_rbf.predict(X_test)
#--- Get prediction probabilites
probabilites_rbf = model_rbf.predict_proba(X_test)
# Get the performance metrics
metrics_rbf = model_rbf.score(X_test, Y_test)
# Print the performance metrics
print('RBF Kernel - Accuracy on the Test Data')
print(metrics_rbf)
from sklearn.metrics import confusion_matrix
conf_mat_rbf = confusion_matrix(Y_test, predictions_rbf)
print('RBF Kernel Confusion Matrix')
print(conf_mat_rbf)

#******************************* POLYNOMIAL Kernal
# Create a NuSVC object with a polynomial kernel
model_poly = NuSVC(kernel='poly', probability=True)
# Fit the classifier to the training data
model_poly.fit(X_train, Y_train)

#----- Get Cross Validation Performances
cv_scores_poly = cross_val_score(model_poly, X_train, Y_train, cv=5, scoring='accuracy')
# Print the cross-validation scores
print("Cross-Validation Scores: Poly Kernal", cv_scores_poly)
print("Mean Cross-Validation Score: Poly Kernal", cv_scores_poly.mean())

# Get the predictions on the test data
predictions_poly = model_poly.predict(X_test)
#--- Get prediction probabilites
probabilites_poly = model_poly.predict_proba(X_test)
# Get the performance metrics
metrics_poly = model_poly.score(X_test, Y_test)
# Print the performance metrics
print('Polynomial Kernel - Accuracy on Test Data' )
print(metrics_poly)
from sklearn.metrics import confusion_matrix
conf_mat_poly = confusion_matrix(Y_test, predictions_poly)
print('Polynomial Kernel Confusion Matrix')
print(conf_mat_poly)

In [ ]:
# Y_test, predictions_linear, probabilites_linear
#Y_test, predictions_rbf, probabilities_rbf
Y_test, predictions_poly, probabilites_poly

In [ ]:
#----- Save Different Performance Matrices - RBF Kernal
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

# Compute precision, accuracy, F1-score, and MCC for the predicted and true labels
precision = precision_score(Y_test, predictions_rbf, average='macro')
accuracy = accuracy_score(Y_test, predictions_rbf)
f1_score = f1_score(Y_test, predictions_rbf, average='macro')
mcc = matthews_corrcoef(Y_test, predictions_rbf)

# Calculate sensitivity and specificity using confusion matrix
cm = confusion_matrix(Y_test, predictions_rbf)

# Calculate sensitivity and specificity for each class
sensitivity = {}
specificity = {}

for i in range(cm.shape[0]):
    tp = cm[i, i]
    fn = sum(cm[i, :]) - tp
    fp = sum(cm[:, i]) - tp
    tn = cm.sum() - (tp + fn + fp)

    sensitivity[i] = tp / (tp + fn)
    specificity[i] = tn / (tn + fp)

# Calculate overall sensitivity and specificity
overall_sensitivity = sum(sensitivity.values()) / len(sensitivity)
overall_specificity = sum(specificity.values()) / len(specificity)

# Print the performance metrics to the console
print("**************** MODEL PERFORMANCE: SuNVC RBF Model****************")
print("Confusion Matrix:")
print(cm)
print("Sensitivity:")
for key, value in sensitivity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Specificity:")
for key, value in specificity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Overall Sensitivity: {:.2f}".format(overall_sensitivity))
print("Overall Specificity: {:.2f}".format(overall_specificity))

print("Precision: {:.2f}".format(precision))
print("Accuracy: {:.2f}".format(accuracy))
print("F1-score: {:.2f}".format(f1_score))
print("MCC: {:.2f}".format(mcc))

In [ ]:
# Save the NuSVC RBF Kernal Results
merged_array_rbf = np.column_stack((Y_test, predictions_rbf))
merged_array_rbf_df = pd.DataFrame(data=merged_array_rbf, columns=['Y_test', 'Y_predicted'])
#print (merged_array_rbf_df)

# Convert predicted probabilities to a NumPy array
#probs_array = probabilities_rbf.detach().numpy()
# Create a DataFrame from the NumPy array
probs_rbf_df = pd.DataFrame(probabilites_rbf, columns=['Class 0', 'Class 1', 'Class 2'])  # Add column names as per your classes
# Print the DataFrame
#print(probs_df)

#-- Get the Drug names from Test Dataset and Scaler values of All variables
drug_names = test_data[['Drug']]

#--- Merge Y_Test_Labels, Y_preds, Y_preds_Proabilites and test Dataset information also
concatenated_rbf_df = pd.concat([merged_array_rbf_df, probs_rbf_df, drug_names, X_test], axis=1)
print (concatenated_rbf_df)

#concatenated_rbf_df.to_csv('ML_results/NuSVC_RBF_Preds.csv', index=False)  # Specify the desired file name and path


# Save the NuSVC Polynomial Kernal Results
merged_array_poly = np.column_stack((Y_test, predictions_poly))
merged_array_poly_df = pd.DataFrame(data=merged_array_poly, columns=['Y_test', 'Y_predicted'])
#print (merged_array_poly_df)

# Convert predicted probabilities to a NumPy array
#probs_array = probabilities_rbf.detach().numpy()
# Create a DataFrame from the NumPy array
probs_poly_df = pd.DataFrame(probabilites_poly, columns=['Class 0', 'Class 1', 'Class 2'])  # Add column names as per your classes
# Print the DataFrame
#print(probs_poly_df)

#-- Get the Drug names from Test Dataset and Scaler values of All variables
drug_names = test_data[['Drug']]

#numpy_array = X_test.numpy()
#X_test_scaler_df = pd.DataFrame(numpy_array, columns = X_test_df.columns)
#X_test_scaler_df

#--- Merge Y_Test_Labels, Y_preds, Y_preds_Proabilites and test Dataset information also
concatenated_poly_df = pd.concat([merged_array_poly_df, probs_poly_df, drug_names, X_test], axis=1)
print (concatenated_poly_df)

#concatenated_poly_df.to_csv('ML_results/NuSVC_Polynomial_Preds.csv', index=False)  # Specify the desired file name and path

In [ ]:
#####Save prediction probabilites figures on the Test DataSet for each Molecule - Using Polynomial Kernel - One time Task
#-- This Code is to Bar Plot the Prediction probabilites for Each Compound Along with the Dot plot all Assay Parameters Associated
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Example dataframe
#df = pd.DataFrame({
#    'Compound': ['A', 'B', 'C'],
#    'Class 1': [0.6, 0.2, 0.4],
#    'Class 2': [0.3, 0.1, 0.5],
#    'Class 3': [0.1, 0.7, 0.1],
#    'Y_test': ['Class 1', 'Class 3', 'Class 1'],
#    'bsep': [100, 300, 400],
#    'lop': [3, -4, 1],
#    'cmax': [150, 20, 100],
#    'glu': [1, 0.1, 0.5],
#    'glu-gal': [20, 70, 50],
#    'fsp3': [1000, 150, 800]
#})

#----------------For Polynomial Kernal
#concatenated_poly_df <- pd.read_csv("ML_results/NuSVC_Polynomial_Preds.csv")
# Reoder the Columns in the concatenated_poly_df file
#df = concatenated_poly_df.loc[:, ['Drug','Class 0','Class 1','Class 2','Y_test','ClogP','BSEP','Glu','Glu_Gal','THLE','HepG2','Fsp3','log10cmax']]

#-------------- For RBF Kernal
#concatenated_rbf_df <- pd.read_csv("ML_results/NuSVC_RBF_Preds.csv")
# Reoder the Columns in the concatenated_poly_df file
df = concatenated_rbf_df.loc[:, ['Drug','Class 0','Class 1','Class 2','Y_test','ClogP','BSEP','Glu','Glu_Gal','THLE','HepG2','Fsp3','log10cmax']]

# Set the width of the bars
bar_width = 0.2

# Set the positions of the bars on the x-axis
bar_positions = np.arange(len(df.columns[1:4]))

# Create a color map for the prediction probabilities
color_map = ['#008b00', '#b8860b', '#b22222']

#--- Get the Value Ranges to Add on the Dot Plot
range_cols = df.columns[5:]
# Set the range for the dot plot
x_range = [df[range_cols].min().min(), df[range_cols].max().max()]
x_min = df[range_cols].values.min()
x_max = df[range_cols].values.max()

# Loop over each compound and save the plot as an image
for i, compound in enumerate(df['Drug']):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    # Bar Plot
    prediction_probs = df.iloc[i, 1:4]
    ax1.bar(bar_positions, prediction_probs, width=bar_width, color=color_map)

    # Customize the plot
    ax1.set_xticks(bar_positions)
    ax1.set_xticklabels(df.columns[1:4])
    ax1.set_xlabel('')
    ax1.set_ylabel('Prediction Probability')
    ax1.set_title(f'Compound name- {compound} - (True Label: {df["Y_test"].iloc[i]})')

    # Dot Plot
    for j, col in enumerate(df.columns[5:]):
        if col in range_cols:
            ax2.plot([df[col].iloc[i]], [j], 'o', color='blue')  # Dot plot

    for j, col in enumerate(range_cols):
        ax2.hlines(y=j, xmin=df[col].min(), xmax=df[col].max(), colors='gray', linestyles='dashed', linewidth=1)

    # Customize the plot
    ax2.set_yticks(range(len(range_cols)))
    ax2.set_yticklabels(range_cols)
    ax2.set_xlabel('Values')
    ax2.set_ylabel('Variables')
    ax2.set_title(f'Compound name {compound}')
    ax2.set_xlim(x_min, x_max)

    # Save the plot as an image
    plt.subplots_adjust(wspace=0.5)
    #---- For Polynomial Kernal
    #plt.savefig(f'ML_results/Predictions_NuSVC/Polynomial/{compound}_plot.png')
    #---- For RBF Kernal
    plt.savefig(f'ML_results/Predictions_NuSVC/RBF/{compound}_plot.png')
    plt.close()

In [ ]:
##### Save prediction probabilites figures on the Test DataSet for each Molecule - Using RBF Kernel -  - One time Task
#-- This Code is to Bar Plot the Prediction probabilites for Each Compound Along with the Dot plot all Assay Parameters Associated
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Example dataframe
#df = pd.DataFrame({
#    'Compound': ['A', 'B', 'C'],
#    'Class 1': [0.6, 0.2, 0.4],
#    'Class 2': [0.3, 0.1, 0.5],
#    'Class 3': [0.1, 0.7, 0.1],
#    'Y_test': ['Class 1', 'Class 3', 'Class 1'],
#    'bsep': [100, 300, 400],
#    'lop': [3, -4, 1],
#    'cmax': [150, 20, 100],
#    'glu': [1, 0.1, 0.5],
#    'glu-gal': [20, 70, 50],
#    'fsp3': [1000, 150, 800]
#})

#concatenated_poly_df <- pd.read_csv("ML_results/NuSVC_Polynomial_Preds.csv")
# Reoder the Columns in the concatenated_poly_df file
df = concatenated_rbf_df.loc[:, ['Drug','Class 0','Class 1','Class 2','Y_test','ClogP','BSEP','Glu','Glu_Gal','THLE','HepG2','Fsp3','log10cmax']]

# Set the width of the bars
bar_width = 0.2

# Set the positions of the bars on the x-axis
bar_positions = np.arange(len(df.columns[1:4]))

# Create a color map for the prediction probabilities
color_map = ['#008b00', '#b8860b', '#b22222']

#--- Get the Value Ranges to Add on the Dot Plot
range_cols = df.columns[5:]
# Set the range for the dot plot
x_range = [df[range_cols].min().min(), df[range_cols].max().max()]
x_min = df[range_cols].values.min()
x_max = df[range_cols].values.max()

# Loop over each compound and save the plot as an image
for i, compound in enumerate(df['Drug']):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    # Bar Plot
    prediction_probs = df.iloc[i, 1:4]
    ax1.bar(bar_positions, prediction_probs, width=bar_width, color=color_map)

    # Customize the plot
    ax1.set_xticks(bar_positions)
    ax1.set_xticklabels(df.columns[1:4])
    ax1.set_xlabel('')
    ax1.set_ylabel('Prediction Probability')
    ax1.set_title(f'Compound name- {compound} - (True Label: {df["Y_test"].iloc[i]})')

    # Dot Plot
    for j, col in enumerate(df.columns[5:]):
        if col in range_cols:
            ax2.plot([df[col].iloc[i]], [j], 'o', color='blue')  # Dot plot

    for j, col in enumerate(range_cols):
        ax2.hlines(y=j, xmin=df[col].min(), xmax=df[col].max(), colors='gray', linestyles='dashed', linewidth=1)

    # Customize the plot
    ax2.set_yticks(range(len(range_cols)))
    ax2.set_yticklabels(range_cols)
    ax2.set_xlabel('Values')
    ax2.set_ylabel('Variables')
    ax2.set_title(f'Compound name {compound}')
    ax2.set_xlim(x_min, x_max)

    # Save the plot as an image
    plt.subplots_adjust(wspace=0.5)
    plt.savefig(f'ML_results/Predictions_NuSVC/RBF/{compound}_plot.png')
    plt.close()

In [ ]:
import os
from PIL import Image
from matplotlib import pyplot as plt

# Directory containing the images
#-- For Polynomial Kernal
#image_dir = '~/Project_DILI/semenova_data/ML_results/Predictions_NuSVC/RBF/'
#-- For RBF Kernal
image_dir = '~/Project_DILI/semenova_data/ML_results/Predictions_NuSVC/RBF/'


# Get a list of image file names in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# Sort the image files by name (if necessary)
image_files.sort()

# Create a list to store the frames of the animation
frames = []

# Load each image and append it to the frames list
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    frames.append(image)

# Create the animation (Play with the Duration to speed up or Slow the animation Speed)
animation = Image.new('RGB', frames[0].size)
#-- For Polynomial Kernal
#animation.save('ML_results/NuSVC_Polynomial_Preds_Animation.gif', format='GIF', append_images=frames[1:], save_all=True, duration=10, loop=0)
#-- For RBF Kernal
animation.save('ML_results/NuSVC_RBF_Preds_Animation.gif', format='GIF', append_images=frames[1:], save_all=True, duration=10, loop=0)

In [ ]:
#-- Feature Importance Calculation using All three models
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Calculate feature importances using permutation importance
result_rbf = permutation_importance(model_rbf, X_test, Y_test, n_repeats=10, random_state=42)
# Get the mean importance score for each feature
importances_rbf = result_rbf.importances_mean
# Sort the feature importances in descending order
sorted_indices_rbf = np.argsort(importances_rbf)[::-1]
# Print the feature importances
for idx in sorted_indices_rbf:
    print(f"Feature {idx}: RBF Model: importance score {importances_rbf[idx]}")

# Create a bar plot of the feature importances
plt.bar(range(X_test.shape[1]), importances_rbf[sorted_indices_rbf])
plt.xticks(range(X_test.shape[1]), sorted_indices_rbf)
plt.title("Feature importances (RBF Model)")
plt.show()
plt.savefig('ML_results/FeaturesImportance_NuSVC_RBF.png', dpi=300)

# Get the names of the features
feature_names = np.array(X_train_df.columns.values)

# Create a bar plot of the feature importances
plt.bar(range(X_test.shape[1]), importances_rbf[sorted_indices_rbf])
plt.xticks(range(X_test.shape[1]), feature_names[sorted_indices_rbf], rotation=90)
plt.title("Feature importances with Names (RBF Model)")
plt.show()
plt.savefig('ML_results/FeaturesImportanceNames_NuSVC_RBF.png', dpi=300)

In [ ]:
##### Optimize Other Parameters of RBF Kernal NuSVC
# Load Libraries
# Data reading in Dataframe format and data preprocessing
import pandas as pd
pd.set_option("display.max_columns", 160)
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset Creation
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV

# Dataset Processing
from sklearn import datasets, linear_model, metrics
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Model Development
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge
from sklearn.svm import SVC

# Model Evaluation
from sklearn.metrics import r2_score, confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.classifier import ClassificationReport, ClassPredictionError
from yellowbrick.regressor import ResidualsPlot, PredictionError

# Feature Importance
import shap

In [ ]:
from sklearn.linear_model import Ridge
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

ridge_pipeline = make_pipeline(Ridge(alpha=10))
ridge_pipeline.fit(X_train, Y_train)
hat_y_test = ridge_pipeline.predict(X_test)

fig, ax = plt.subplots(1)
sns.regplot(data=None, x=Y_test, y=hat_y_test, line_kws={"color":"black", "linestyle":"dotted"})
plt.plot()
plt.savefig('ML_results/regression_plot.png', dpi=300)
ax.set(xlabel=r"y", ylabel=r"y'")
test_r2_score_ = r2_score(Y_test, hat_y_test)
print(f"R2 score: {test_r2_score_}")

In [ ]:
#**************************************************************************
#------ Alternative to Check the Brier Score for Each Class
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

# Create an uncalibrated NuSVC model
model_rbf = NuSVC(kernel='rbf', probability=True)

# Fit the model to the training data
model_rbf.fit(X_train, Y_train)

# Get the predicted probabilities on the test data
probabilities = model_rbf.predict_proba(X_test)

# Calibrate the predicted probabilities using CalibratedClassifierCV
calibrated_model_rbf = CalibratedClassifierCV(model_rbf, cv=5)
calibrated_model_rbf.fit(X_train, Y_train)
calibrated_probabilities = calibrated_model_rbf.predict_proba(X_test)

# Compute the Brier score for each class
brier_scores = []
for class_idx in range(probabilities.shape[1]):
    true_labels = np.zeros_like(Y_test)
    true_labels[Y_test == class_idx] = 1
    uncalibrated_brier_score = brier_score_loss(true_labels, probabilities[:, class_idx])
    calibrated_brier_score = brier_score_loss(true_labels, calibrated_probabilities[:, class_idx])
    brier_scores.append((uncalibrated_brier_score, calibrated_brier_score))

# Print the Brier scores for each class
for class_idx, (uncalibrated_score, calibrated_score) in enumerate(brier_scores):
    print(f"Class {class_idx}:")
    print(f"  Uncalibrated Brier Score: {uncalibrated_score}")
    print(f"  Calibrated Brier Score: {calibrated_score}")

### 4. POLR Method as used by Semenova Et Al.

In [ ]:
# design matrices
from patsy import dmatrix

#- Number of columns increased likely due to the addition of the quadratic terms and interactions specified in the dmatrix formula string
# Quadratic terms for these variables in the resulting design matrix. This means that for each of these variables, two new columns are added to the design matrix: one with the original variable values, and one with the squared variable values.
# Additionally, the formula string includes the + operator between each of the variables, which instructs dmatrix to include interaction terms between the variables in the resulting design matrix. This means that for each pair of variables, a new column is added to the design matrix with the product of the two variable values.

X_train_new = pd.DataFrame(dmatrix("0 + (ClogP + BSEP + THLE + Glu + Glu_Gal + HepG2 + Fsp3)**2 + log10cmax", X_train))
X_test_new = pd.DataFrame(dmatrix("0 + (ClogP + BSEP + THLE + Glu + Glu_Gal + HepG2 + Fsp3)**2 + log10cmax", X_test))

y_train_new = train_data['dili_sev'].values #--- Y_shared-1 was used while creating the model
y_test_new = test_data['dili_sev'].values

In [ ]:
## ---------------------------------------------------------
## Define and fit model
## ---------------------------------------------------------

N = X_train_new.shape[0]
p = X_train_new.shape[1]
X_train_new = X_train_new.values
#y_train.reshape((96,1));
sigma_prior = 1

# share variable to enable predictions
X_shared = theano.shared(X_train_new)
y_shared = theano.shared(y_train_new)

with pm.Model() as model:

    mu_beta = pm.Normal('mu_beta', mu=0, sd=2)
    sd_beta = pm.HalfNormal('sd', sd=sigma_prior)
    beta = pm.Laplace('beta', mu=mu_beta, b=sd_beta, shape=p)

    cutpoints = pm.Normal("cutpoints", mu=[-0.001,0], sd=20, shape=2,
                           transform=pm.distributions.transforms.ordered)

    lp = pm.Deterministic('lp', pm.math.dot(X_shared, beta))

    y_obs = pm.OrderedLogistic("y_obs", eta=lp, cutpoints=cutpoints, observed=y_shared-1)

    trace = pm.sample(draws = 20000) #Try changing pm.sample(samples=20000) to pm.sample(draws=20000) and see if it resolves the issue.

In [ ]:
#import pickle

#***************** Save Trace and Model
#----------Save the trace
#with open("Final_models/SemenovaData_POLR_trace.pkl", "wb") as file:
#    pickle.dump(trace, file)
#----------Save the model
#with open("Final_models/SemenovaData_POLR_model.pkl", "wb") as file:
#    pickle.dump(model, file)

#***************** Load Trace and Model
#------------ Load the trace
#with open("Final_models/SemenovaData_POLR_trace.pkl", "rb") as file:
#    trace = pickle.load(file)

#------------ Load the model
#with open("Final_models/SemenovaData_POLR_model.pkl", "rb") as file:
#    model = pickle.load(file)

In [ ]:
import arviz as az
#az.plot_trace(trace)

with model:
    az.plot_trace(trace)

In [ ]:
# Summary
pm.summary(trace).round(2)

In [ ]:
## ---------------------------------------------------------
## predict for training data
## ---------------------------------------------------------
post_pred_train = pm.sample_posterior_predictive(trace, samples=5000, model=model)

# exctract predictions
y_pred_train_df = post_pred_train['y_obs']
# Get predicted probabilities
y_pred_probs_mean = np.mean(post_pred_train['y_obs'], axis=0)

In [ ]:
# Summarize predictions
y_pred_train_df
y_pred_train_df.shape # 5000 times 147 Columns (Data) - For Each compound get which class got majority votes
y_pred_train_df_df = pd.DataFrame(y_pred_train_df)

import pandas as pd
import numpy as np
from scipy.stats import mode

# Assuming you have a NumPy array called 'y_pred_train_df' with shape (5000, 147)
# Each column contains three predicted labels

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(y_pred_train_df_df)

# Get the label with the majority of votes for each column
majority_labels = np.asarray(mode(df, axis=0)[0])[0]

# Print the majority labels
print(majority_labels)

# Just check
y_pred_train_df[0].value_counts()

In [ ]:
Y_train_updated = y_train_new.astype(int) - 1 # Y-actual lables were 1,2,3 but in model 0,1,2 was used
train_actual_pred = np.column_stack((Y_train_updated, majority_labels))
train_actual_pred_df = pd.DataFrame(data=train_actual_pred, columns=['Train_actual_label', 'Train_pred_label'])
train_actual_pred_df
train_actual_pred_df.to_csv('ML_results/POLR_trainData_Perform.csv', index=False)  # Specify the desired file name and path

In [ ]:
#----- Save Different Performance Matrices
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

# Compute precision, accuracy, F1-score, and MCC for the predicted and true labels
precision = precision_score(Y_train_updated, majority_labels, average='macro')
accuracy = accuracy_score(Y_train_updated, majority_labels)
f1_score = f1_score(Y_train_updated, majority_labels, average='macro')
mcc = matthews_corrcoef(Y_train_updated, majority_labels)

# Calculate sensitivity and specificity using confusion matrix
cm = confusion_matrix(Y_train_updated, majority_labels)

# Calculate sensitivity and specificity for each class
sensitivity = {}
specificity = {}

for i in range(cm.shape[0]):
    tp = cm[i, i]
    fn = sum(cm[i, :]) - tp
    fp = sum(cm[:, i]) - tp
    tn = cm.sum() - (tp + fn + fp)

    sensitivity[i] = tp / (tp + fn)
    specificity[i] = tn / (tn + fp)

# Calculate overall sensitivity and specificity
overall_sensitivity = sum(sensitivity.values()) / len(sensitivity)
overall_specificity = sum(specificity.values()) / len(specificity)

# Print the performance metrics to the console
print("**************** MODEL PERFORMANCE: POLR on Training Dataset ****************")
print("Confusion Matrix:")
print(cm)
print("Sensitivity:")
for key, value in sensitivity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Specificity:")
for key, value in specificity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Overall Sensitivity: {:.2f}".format(overall_sensitivity))
print("Overall Specificity: {:.2f}".format(overall_specificity))

print("Precision: {:.2f}".format(precision))
print("Accuracy: {:.2f}".format(accuracy))
print("F1-score: {:.2f}".format(f1_score))
print("MCC: {:.2f}".format(mcc))

In [ ]:
#---- Now do the Predictions on Test Data
# ### Predict for test data
X_shared.set_value(X_test_new)
y_shared.set_value([0, 0]) # dummy values
post_pred_test = pm.sample_posterior_predictive(trace, samples=5000, model=model)

y_pred_test_df = post_pred_test['y_obs']
y_pred_test_df.shape

In [ ]:
# posterior prediction for each category
def probs(y_pred_df, ind):
    y_pred_ind = y_pred_df[:,ind]
    p = [sum(y_pred_ind == 0)/len(y_pred_ind), sum(y_pred_ind == 1)/len(y_pred_ind), sum(y_pred_ind == 2)/len(y_pred_ind)]
    return p

# compute posterior probabilities for a selected drug
print(probs(y_pred_test_df, 0))
print(probs(y_pred_test_df, 1))
print(probs(y_pred_test_df, 2))

In [ ]:
# Summarize predictions on The Test DataSet
y_pred_test_df
y_pred_test_df.shape # 5000 times 37 Columns (Data) - For Each compound get which class got majority votes
y_pred_test_df_df = pd.DataFrame(y_pred_test_df)

import pandas as pd
import numpy as np
from scipy.stats import mode

# Assuming you have a NumPy array called 'y_pred_train_df' with shape (5000, 147)
# Each column contains three predicted labels

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(y_pred_test_df_df)

# Get the label with the majority of votes for each column
pred_majority_labels = np.asarray(mode(df, axis=0)[0])[0]

# Print the majority labels
print(pred_majority_labels)

# Just check
y_pred_test_df_df[0].value_counts()

In [ ]:
Y_test_updated = y_test_new.astype(int) - 1 # Y-actual lables were 1,2,3 but in model 0,1,2 was used
test_actual_pred = np.column_stack((Y_test_updated, pred_majority_labels))
test_actual_pred_df = pd.DataFrame(data=test_actual_pred, columns=['Test_actual_label', 'Test_pred_label'])
test_actual_pred_df

In [ ]:
import pandas as pd

# Initialize an empty DataFrame
probabilities_df = pd.DataFrame()

# Iterate over the instances and extract probabilities
class_index = 0  # Index of the desired class
for i in range(37):
    instance_probs = probs(y_pred_test_df, i)
    probabilities_df = probabilities_df.append(pd.Series(instance_probs), ignore_index=True)

# Set column names for the probabilities DataFrame
probabilities_df.columns = ['Class 1', 'Class 2', 'Class 3']

# Print the probabilities DataFrame
print(probabilities_df)

In [ ]:
# Save Prediction Results of POLR model on Test Dataset
# test_actual_pred_df - has actual and predicted lables
# print (test_actual_pred_df)
# probabilities_df - has all three probabilities
# print(probabilities_df)
#-- Get the Drug names from Test Dataset and Scaler values of All variables
drug_names = test_data[['Drug']]

#--- Merge Y_Test_Labels, Y_preds, Y_preds_Proabilites and test Dataset information also
POLR_concatenated_df = pd.concat([test_actual_pred_df, probabilities_df, drug_names, X_test], axis=1)
print (POLR_concatenated_df)

POLR_concatenated_df.to_csv('ML_results/POLR_testData_Predictions.csv', index=False)  # Specify the desired file name and path

In [ ]:
#----- Check Different Performance Matrices on Test Data Set
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

# Compute precision, accuracy, F1-score, and MCC for the predicted and true labels
precision = precision_score(Y_test_updated, pred_majority_labels, average='macro')
accuracy = accuracy_score(Y_test_updated, pred_majority_labels)
f1_score = f1_score(Y_test_updated, pred_majority_labels, average='macro')
mcc = matthews_corrcoef(Y_test_updated, pred_majority_labels)

# Calculate sensitivity and specificity using confusion matrix
cm = confusion_matrix(Y_test_updated, pred_majority_labels)

# Calculate sensitivity and specificity for each class
sensitivity = {}
specificity = {}

for i in range(cm.shape[0]):
    tp = cm[i, i]
    fn = sum(cm[i, :]) - tp
    fp = sum(cm[:, i]) - tp
    tn = cm.sum() - (tp + fn + fp)

    sensitivity[i] = tp / (tp + fn)
    specificity[i] = tn / (tn + fp)

# Calculate overall sensitivity and specificity
overall_sensitivity = sum(sensitivity.values()) / len(sensitivity)
overall_specificity = sum(specificity.values()) / len(specificity)

# Print the performance metrics to the console
print("**************** MODEL PERFORMANCE: POLR on Training Dataset ****************")
print("Confusion Matrix:")
print(cm)
print("Sensitivity:")
for key, value in sensitivity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Specificity:")
for key, value in specificity.items():
    print("Class {}: {:.2f}".format(key, value))
print("Overall Sensitivity: {:.2f}".format(overall_sensitivity))
print("Overall Specificity: {:.2f}".format(overall_specificity))

print("Precision: {:.2f}".format(precision))
print("Accuracy: {:.2f}".format(accuracy))
print("F1-score: {:.2f}".format(f1_score))
print("MCC: {:.2f}".format(mcc))

### `Final BNN Code is in the seperate File (BNN_Check.ipynb). That BNN model was developed using the same parameters used in the origional paper.`